In [1]:
import torch
import random
import datetime
import pandas as pd
import numpy as np
import os

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems
from src.embeddings import RecsysEmbedding

experiment_name = 'LogReg'
device = 'cuda:0'
seed = 7331
pkl_path = '../data/'


random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Модель

In [2]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, embedding, output_dim=1):
        super().__init__()
        self.embedding = embedding
        self.linear = torch.nn.Linear(2 * embedding.embedding_dim, output_dim)
    
    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)

        features = torch.cat(
            [
                item_embs,
                user_embs[:, :, None, :].repeat(1, 1, item_embs.size(-2), 1)
            ],
            dim = -1
        )
        return self.linear(features).squeeze(-1)
    

d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)
emb = RecsysEmbedding(d.n_items, dummy_matrix, embeddings='explicit')

for batch in dummy_loader:
    break

    
model = LogisticRegression(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural'),
    output_dim=1
).to('cpu')

model(batch)


3it [00:00, 4016.25it/s]

biulding affinity matrix...


tensor([[[ 0.6851, -0.5144, -0.0075],
         [-0.3600, -0.8669,  0.0244]],

        [[ 0.3116,  0.3116,  0.3769],
         [ 0.3769,  0.3769,  0.3769]]], grad_fn=<SqueezeBackward1>)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [3]:
d = DummyData()
dummy_loader, dummy_svd = get_dummy_data(d)

for embeddings in ['explicit', 'neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = LogisticRegression(
        RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural'),
        output_dim=1
    ).to('cpu')

    _, metrics = train(model, 
       dummy_loader, dummy_loader, dummy_loader, encoder=dummy_svd if embeddings == 'svd' else None,
       device=device, lr=1e-3, num_epochs=5000, 
       silent=True
    )


3it [00:00, 2544.57it/s]

biulding affinity matrix...

Evaluating LogReg with explicit embeddings



train... loss:0.6516387462615967:   0%|                                                                                      | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.0, 'roc-auc': 1.0, 'accuracy': 0.75}


train... loss:0.6474779844284058:   0%|                                                                              | 1/5000 [00:00<20:40,  4.03it/s]

...

Test update: epoch: 4 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 1.0 | treshold: 0.33


train... loss:0.723713219165802:   0%|                                                                               | 6/5000 [00:01<18:18,  4.55it/s]

Test update: epoch: 5 |accuracy: 0.5 | f1: 0.5 | auc: 1.0 | treshold: 0.34


train... loss:0.723713219165802:   0%|                                                                               | 6/5000 [00:01<20:32,  4.05it/s]

Test update: epoch: 6 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.42000000000000004


# ContentWise

In [4]:
content_wise_results = []
c = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
c_train_loader, c_val_loader, c_test_loader, c_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(c, batch_size=150)
len(c_train_loader), len(c)

(108, 20216)

In [5]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = LogisticRegression(
        RecsysEmbedding(train_num_items, c_train_user_item_matrix, embeddings='neural').to('cpu'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       c_train_loader, c_val_loader, c_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)
    
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')


Evaluating LogReg with svd embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.18089768290519714, 'roc-auc': 0.5208644270896912, 'accuracy': 0.4691844880580902}


train... loss:41.33676525950432:   0%|                                                                            | 2/5000 [00:24<20:00:56, 14.42s/it]

Test update: epoch: 1 |accuracy: 0.44251054525375366 | f1: 0.20086047053337097 | auc: 0.5975727438926697 | treshold: 0.06999999999999999


...

train... loss:31.41856551170349:   0%|▏                                                                           | 12/5000 [01:13<8:30:57,  6.15s/it]

Test update: epoch: 11 |accuracy: 0.7334667444229126 | f1: 0.2627980709075928 | auc: 0.687035322189331 | treshold: 0.12


train... loss:31.257891476154327:   0%|▎                                                                          | 24/5000 [02:25<8:23:03,  6.07s/it]

Test update: epoch: 23 |accuracy: 0.7932145595550537 | f1: 0.2644057273864746 | auc: 0.6885620355606079 | treshold: 0.14


train... loss:31.247106552124023:   0%|▎                                                                          | 24/5000 [02:31<8:43:06,  6.31s/it]


# RL4RS

In [6]:
rl4rs_results = []
r = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
r_train_loader, r_val_loader, r_test_loader, r_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(r, batch_size=20000)
len(r_train_loader), len(r)

(2, 45942)

In [7]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = LogisticRegression(
        RecsysEmbedding(train_num_items, r_train_user_item_matrix, embeddings='neural'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       r_train_loader, r_val_loader, r_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=50,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)
    
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')


Evaluating LogReg with neural embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.5323517322540283, 'roc-auc': 0.5173546075820923, 'accuracy': 0.49771490693092346}


train... loss:1.4511416554450989:   0%|                                                                           | 1/5000 [00:36<50:51:24, 36.62s/it]

Test update: epoch: 0 |accuracy: 0.6395599246025085 | f1: 0.7801215648651123 | auc: 0.5230585336685181 | treshold: 0.17


...

train... loss:0.8541781604290009:   7%|█████▎                                                                   | 368/5000 [35:48<16:47:47, 13.05s/it]

Test update: epoch: 367 |accuracy: 0.8332728743553162 | f1: 0.875029444694519 | auc: 0.9129464626312256 | treshold: 0.6


train... loss:0.8549995124340057:   7%|█████▍                                                                   | 369/5000 [36:07<19:07:02, 14.86s/it]

Test update: epoch: 368 |accuracy: 0.836077868938446 | f1: 0.87483149766922 | auc: 0.9129375219345093 | treshold: 0.65


train... loss:0.8533502817153931:   8%|██████                                                                    | 413/5000 [47:38<8:49:12,  6.92s/it]
